![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>最小二乗法<br></center></h1>

このノートブックでは、MuJoCo Pythonライブラリに含まれるボックス制約付き非線形最小二乗最適化を行うユーティリティ関数について説明します。理論的背景を説明し、実装の詳細を述べ、使用例を示します。
<a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/least_squares.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="120" align="center"/></a>

<!--

Copyright 2022 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

-->

# すべてのインポート

In [0]:
# MuJoCoのインストール
!pip install mujoco
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'GPUと通信できません。'
      'GPU対応のColabランタイムを使用していることを確認してください。'
      'ランタイムメニューから「ランタイムのタイプを変更」を選択してください。')

# glvndがNvidia EGLドライバーを検出できるようにICD設定を追加
# 通常はNvidiaドライバーパッケージの一部としてインストールされますが、Colab
# カーネルはAPT経由でドライバーをインストールしないため、ICDが欠落しています。
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")
#
# GPUレンダリングを使用するようにMuJoCoを設定（GPUが必要）
print('GPU レンダリングを使用するための環境変数を設定中:')
%env MUJOCO_GL=egl

try:
  print('インストールが成功したか確認中:')
  import mujoco
  from mujoco import minimize
  from mujoco import rollout
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'インストール中に問題が発生しました。詳細については上記のシェル出力を'
      '確認してください。\n'
      'ホストされたColabランタイムを使用している場合は、ランタイムメニューから'
      '「ランタイムのタイプを変更」を選択してGPUアクセラレーションを有効にしてください。')
print('MuJoCoのインストールに成功しました。')

print('mediapyをインストール中:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
#
from IPython.display import clear_output
clear_output()

# その他のインポート
import mediapy as media
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from typing import Tuple, Optional, Union


# 背景

まず簡単な入門から始めます。この内容をご存知の方は、このセクションをスキップしてください。

## ニュートン法
非線形最適化（ここではまず制約なしの場合）とは、 **目的関数** $f(x): \mathbb R^n \rightarrow \mathbb R$（ここでは滑らかな関数と仮定）を最小化するベクトル **決定変数** $x \in \mathbb R^n$ の値を見つけることです。これは以下のように書かれます。 $$x^* = \arg \min_x f(x).$$

2次最適化、すなわちニュートン法は、$f(x_k)$ の値を減少させる候補列 $x_k$ を見つける反復手続きで、最小値に収束するまで繰り返します。各反復で $f(\cdot)$ の局所的な1次および2次導関数を計測します。これらはそれぞれ **勾配** ベクトル $g = \nabla f(x_k)$ および **ヘッセ行列** $H =  \nabla^2 f(x_k)$ と呼ばれます。これらにより $f(x_k)$ の局所的な2次近似が得られます：
$$
f(x_k + \delta x) \approx f(x_k) + \delta x^T g + \frac{1}{2} \delta x^T H\delta x
$$
$\delta x$ で微分して0とおき解くと、2次関数の最小値にジャンプでき、次の候補が得られます：
$$
x_{k+1} = x_k + \delta x = x_k - H^{-1}g
$$

## Levenberg-Marquardt正則化
上記の形式のままのニュートン法は、良いアイデアではありません。候補 $x_{k}$ が収束する保証はなく、実際に目的関数が減少するという保証すらありません。これは以下の理由によります：
1. 局所的な2次近似が非常に悪い可能性がある（小さな近傍でのみ有効）。
2. 近似が良い場合でも、$H$ が正定値ではなく負定値（または不定値）である可能性がある。この場合、2次関数の *最小値* ではなく *最大値*（または *鞍点*）にジャンプしてしまい、状況がさらに悪化する。

Levenberg-Marquardt（LM）正則化は、正則化パラメータ $\mu \ge 0$ を導入することでこれらの問題を両方とも解決します：
$$
\delta x_{LM} = - (H + \mu I)^{-1}g
$$
$\mu=0$ の場合、古典的なニュートンステップが得られます。$\mu$ が十分大きく $\mu I \gg H$ の場合、$\delta x \approx -\tfrac{1}{\mu}g$：小さな勾配ステップとなり、十分大きな $\mu$ に対して目的関数の減少が保証されます。以下のセルを実行すると、LM正則化の可視化が表示されます。単純な2次関数と初期推定値の場合、
$$
f(x) = \frac{1}{2}x^T \begin{pmatrix} 2 & 0.9\\ 0.9 & 1 \end{pmatrix} x,
\qquad x_0 = \begin{pmatrix} -0.9\\ 0.5 \end{pmatrix},
$$
パラメータを対数的に変化させて $\mu = 10^{-4} \ldots 10^2$ とした場合の $x_1(\mu)$ の軌跡をプロットします。

In [0]:
#@title LM曲線の可視化

# 問題設定
H = np.array([[2, 0.9], [0.9, 1]])
x0 = np.array((-0.9, 0.5))

# 対数的に等間隔なmu正則化値
lm_values = 10**np.linspace(-4, 2, 40)

# 次の候補の列 x1(mu)
g0 = H.dot(x0)
dx = np.asarray([-np.linalg.inv(H + mu*np.eye(2)).dot(g0) for mu in lm_values])
x1 = x0 + dx

# 関数値のグリッド、等高線プロット
res = 400
X,Y = np.meshgrid(np.linspace(-1, 1, res),
                  np.linspace(-1, 1, res))
Z = 0.5 * (H[0,0]*X**2 + H[1,1]*Y**2 + 2*H[1,0]*X*Y)
fig = plt.figure(figsize=(10,10))
contours = np.linspace(0, np.sqrt(Z.max()), 20)**2
plt.contour(X, Y, Z, contours, linewidths=0.5)

# x0、実際の最小値、x1(mu)を描画
point_spec = {'markersize':10, 'markeredgewidth':2, 'fillstyle':'none',
              'marker':'o', 'linestyle':'none'}
curve_spec = {'markersize':1.5, 'linewidth':1, 'color':'red'}
plt.plot(x0[0], x0[1], color='grey', **point_spec)
plt.plot(0, 0, color='pink', **point_spec)
plt.plot(x1[:,0], x1[:,1], '-o', **curve_spec)

# 図の仕上げ
plt.title('Levenberg–Marquardt曲線: $x_1(\mu),\; \mu = 10^{-4} \ldots 10^2$')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['$x_0$', '最小値', 'LM軌跡'])
plt.rc('font', family='serif')

上のプロットでは「LM曲線」が表示されています。小さな $\mu$ では2次関数の最小値から始まり、$\mu$ が大きくなるにつれて $x_0$ に向かって曲がり、勾配に沿って $x_0$ に近づいていきます。Powellのdog-leg軌跡は、LM曲線の区分線形近似と考えることができます。

重要なのは、曲線の両端で点が密集していることです。曲線が「有用」で意味のある探索を可能にする $\mu$ の範囲は、6桁にわたって変化させていることを考えると、かなり狭いです。これは、$\mu$ の慎重な制御が収束の効率にとって重要であることを意味しています。

## ボックス制約
探索空間を制限することで、追加の安定性と頑健性を達成できます。重要な制約の種類はボックス制約です：
$$
\begin{aligned}
x^* &= \arg \min_x f(x)\\
\textrm{s.t.} &\quad l \preccurlyeq x \preccurlyeq u
\end{aligned}
$$
ここで $l$ と $u$ はそれぞれ下限と上限のベクトルで、不等式 $l \preccurlyeq x \preccurlyeq u$ は要素ごとに読みます。最小化する $\delta x$ を解くことは線形システムを解くよりもやや複雑になり、この最適化手法は [逐次二次計画法](https://en.wikipedia.org/wiki/Sequential_quadratic_programming)（SQP）と呼ばれます。幸いなことに、MuJoCoにはボックス制約付きQPの効率的なソルバーである [mju_boxQP](https://mujoco.readthedocs.io/en/latest/APIreference/APIfunctions.html#mju-boxqp) 関数があります。

## 最小二乗法とガウス・ニュートンヘッセ行列

構造化された目的関数によりさらなる簡略化が達成されます： $f(x) = \frac{1}{2} r(x)^T\cdot r(x)$。これは *最小二乗* 目的関数と呼ばれます。ベクトル関数 $r(x): \mathbb R^n \rightarrow \mathbb R^m$ は **残差** と呼ばれ、この最適化問題は *非線形最小二乗法* と呼ばれます。統計学のバックグラウンドをお持ちの方にとっては、最小二乗問題はガウス事後分布 $x^* = \arg \max_x p(x)$（$p(x) \sim e^{-\frac{1}{2}r(x)^Tr(x)}$）の **最尤推定** として再解釈できます。最小二乗構造がなぜ有益かを理解するために、目的関数の勾配とヘッセ行列を考えます。$J$ を残差の $n \times m$ **ヤコビ** 行列 $J = \nabla r(x)$ とすると、
$$
\begin{align}
g &= J^Tr\\
H &= J^TJ + \nabla J \cdot r
\end{align}
$$
ヘッセ行列 $H$ の *ガウス・ニュートン* 近似は、第2項を落とすことで得られます：$H \approx H_{GN}= J^TJ$。この近似には2つの明らかな利点があります：
1. 近似ヘッセ行列 $H_{GN}$ は行列の2乗であるため、負定値にはなり得ず、少なくとも半正定値です。任意の $\mu \gt 0$ に対して、LM正則化行列 $H_{GN} + \mu I$ は対称正定値（SPD）であることが保証されます。
2. $H_{GN}$ を計算するにはヤコビアン $J$ のみが必要です。つまり、1次導関数のみで2次最適化の利点が得られます。

## 最小ノルムへの一般化

上記の最小二乗問題は以下のように一般化できます。2次ノルム $\frac{1}{2} r(x)^T\cdot r(x)$ の代わりに、他の滑らかで凸なノルム $f(x) = n(r(x))$ を使用できます。$\nabla n = \frac{\partial n}{\partial r}$ と $\nabla^2 n = \frac{\partial^2 n}{\partial r^2}$ をそれぞれ $n$ の $r$ に関する勾配とヘッセ行列とすると、
$$
\begin{align}
g &= J^T\cdot\nabla n\\
H_{GN} &= J^T\cdot \nabla^2 n \cdot J
\end{align}
$$
2次ノルムの場合、$\nabla (\frac{1}{2}r^T\cdot r) = r$ および $\nabla^2 (\frac{1}{2}r^T\cdot r) = I_m$ となるため、これらの式が上記のものに帰着することは容易に確認できます。

以上で背景セクションは完了です。ここからは、このノートブックの残りで探求する関数を正確に説明できます。

# `minimize.least_squares`

`mujoco` Pythonライブラリの `minimize.least_squares` 関数は、ボックス制約付き非線形最小二乗問題を解きます。ガウス・ニュートンヘッセ近似を使用し、LMパラメータ $\mu$ を調整しながらLevenberg-Marquardt探索ステップを取ります。

## 動機と仮定

`minimize.least_squares()` 関数は2つの問題に動機づけられています：

- **システム同定**（sysID）問題が主な用途です。sysIDは、実システムの挙動をよりよくマッチさせるためにシミュレーションモデルのパラメータを見つけることを含みます。前セクションの表記法では、この問題は決定変数 $x$ が同定したいモデルパラメータに対応し、残差 $r$ が計測されたセンサー値とシミュレーションされたセンサー値の差に対応する最小二乗問題として定式化できます。例を含む完全なsysIDチュートリアルは近日公開予定です。
- **逆運動学**（IK）問題。この場合、決定変数は関節角度であり、残差はエンドエフェクタの姿勢と目標姿勢の差です。IK問題は設計上の中心ではありませんでしたが、我々のコードで非常に効率的に解くことができます。

この動機は、以下の仮定と設計上の選択につながりました。

1. 決定変数の次元は小さい：$\dim(x) \lesssim 100$。コレスキー分解（QPソルバーの主要コスト）は $\dim(x)$ に対して3乗でスケールしますが、このサイズでは非常に高速です。
2. 残差ベクトル $r$ の次元（sysIDの場合、計測されたセンサー値の数に対応）ははるかに大きくなり得ます。
3. 残差の各評価はシミュレートされたセンサー値を取得するために物理のロールアウトを伴うため、$r(x)$ の計算は最適化の中で最もコストの高い部分です。
4. 解析的ヤコビアン $J = \frac{\partial r}{\partial x}$ は通常利用できず、 **有限差分** で取得する必要があります。
5. $x$ のセマンティクスにより、ボックス制約で通常十分です（例えば、質量や摩擦係数は負にできず、関節角度は限界を超えてはいけません）。
6. 実装は効率的でありながら読みやすくあるべきです。[least_squares関数](https://github.com/google-deepmind/mujoco/blob/main/python/mujoco/minimize.py) は約250行のコードです。

関数のdocstringを確認し、いくつかの実装ノートを議論しましょう。

In [0]:
print(minimize.least_squares.__doc__)

## 実装ノート

1. 残差関数はベクトル化されている必要があります：列ベクトル $x$ を取って残差 $r(x)$ を返すだけでなく、$n\times k$ 行列 $X$ を受け取り、$m\times k$ 行列 $R$ を返す必要があります。ベクトル化された形式は内部の有限差分実装で使用され、残差関数の実装内でマルチスレッドを使用して最小化を高速化するために利用できます。
1. 制約は `None` または $x$ のすべての次元に対して完全に指定する必要があります。
1. `jacobian` コールバックはユーザーが提供でき、そうでない場合は有限差分で計算されます。このコールバックは、ユーザーが解析的ヤコビアンを知っている場合（sysIDの文脈では非常にまれ）のためではなく、ユーザーが独自のマルチスレッド有限差分コールバックを実装したい場合に利用可能です。
1. 制約を超えないように選択される自動の前進/後退差分と、オプションの中心差分。有限差分のイプシロン `eps` は、提供されている場合、制約の範囲でスケーリングされます。
1. 終了基準は小さなステップサイズ $||\delta x|| < \textrm{tol}$ に基づいています。
1. [Bazaraa et-al.](https://onlinelibrary.wiley.com/doi/book/10.1002/0471787779) に記述されている、シンプルだが効果的な $\mu$ 探索戦略を使用しています。バックトラッキングによる $\mu$ の増加は *慎重* であり、十分な減少が見つかる最小の $\mu$ を見つけようとします。$\mu$ の減少は *積極的* であり、局所最小値への高速な2次収束を可能にします。
1. ユーザーはオプションで2次ノルム（デフォルト）とは異なる `norm` を提供できます。これについては以下でより詳細に説明します。

# 簡単な例

In [0]:
# @title プロットユーティリティ
def plot_2D(residual, name, plot_range, true_minimum, trace=None, bounds=None):
  n = 400
  x_range, y_range = plot_range
  x_grid = np.linspace(x_range[0], x_range[1], n)
  y_grid = np.linspace(y_range[0], y_range[1], n)
  X, Y = np.meshgrid(x_grid, y_grid)

  R = residual(np.stack((X, Y)))
  Z = 0.5 * np.sum(R**2, axis=0)

  fig = plt.figure(figsize=(10, 10))
  cntr_levels = np.linspace(0, np.log1p(Z.max()), 30)
  plt.contour(X, Y, np.log1p(Z), cntr_levels, linewidths=0.5)
  plt.title(name)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.rc('font', family='serif')

  # 制約を描画
  hbounds = None
  if bounds is not None:
    lower, upper = bounds
    width = upper[0] - lower[0]
    height = upper[1] - lower[1]
    rect = Rectangle(lower, width, height, edgecolor='blue', facecolor='none',
                     fill=False, lw=1)
    hbounds = fig.axes[0].add_patch(rect)

  # 大域的最小値、初期点を描画
  point_spec = {'markersize': 10, 'markeredgewidth': 2, 'fillstyle': 'none',
                'marker': 'o', 'linestyle': 'none'}
  curve_spec = {'marker': 'o', 'markersize': 2, 'linewidth': 1.5,
                'color': 'red', 'alpha':0.5}
  final_spec = {'marker': 'o', 'markersize': 4, 'color': 'green',
                'linestyle': 'none'}
  hmin = plt.plot(true_minimum[0], true_minimum[1], color='pink', **point_spec)

  def add_curve(t):
    x0 = plt.plot(t[0, 0], t[0, 1], color='grey', **point_spec)
    traj = plt.plot(t[:, 0], t[:, 1], **curve_spec)
    final = plt.plot(t[-1, 0], t[-1, 1], **final_spec)
    return x0, traj, final

  if trace is not None:
    if isinstance(trace, list):
      for t in trace:
        x0, traj, final = add_curve(t)
    else:
      x0, traj, final = add_curve(trace)

  handles = [x0[0], traj[0], hmin[0], final[0]]
  labels = ['$x_0$', '最適化の軌跡', '真の最小値', '解']
  if hbounds is not None:
    handles.append(hbounds)
    labels.append('ボックス制約')
  plt.legend(handles, labels)

  plt.show()

## Rosenbrock関数

古典的な [Rosenbrokテスト関数](https://en.wikipedia.org/wiki/Rosenbrock_function) は最小二乗形式で書くことができます：

$f(x,y)=r^Tr,$ 残差は
$r(x,y)=\begin{pmatrix} 1-x \\ 10\cdot(y-x^2) \end{pmatrix}$ です。

ここで2Dの例のセクションでは、$x, y$ を決定変数の2つの座標を表すために使用しています。 `minimize.least_squares` が最小値を見つける様子を見てみましょう：

In [0]:
# Rosenbrock関数を最小化
def rosenbrock(x):
  return np.stack([1 - x[0, :], 10 * (x[1, :] - x[0, :] ** 2)])

x0 = np.array((0.0, 0.0))
x, rb_trace = minimize.least_squares(x0, rosenbrock);

In [0]:
#@title Rosenbrock関数の解を可視化

plot_range = (np.array((-0.5, 1.5)), np.array((-1.5, 2.)))
minimum = (1, 1)
points = np.asarray([t.candidate for t in rb_trace])
plot_2D(rosenbrock, 'Rosenbrock関数', plot_range, minimum, trace=points)

## Beale関数

もう少し複雑な例として [Beale関数](https://en.wikipedia.org/wiki/Test_functions_for_optimization#Test_functions_for_single-objective_optimization) があります：

$f(x,y)=r^Tr,$ 残差は
$r(x,y)=\begin{pmatrix} 1.5-x+xy \\ 2.25-x+xy^2 \\ 2.625-x+xy^3 \end{pmatrix}$ です。

In [0]:
#@title Beale関数の最小化と可視化
def beale(x):
  return np.stack((1.5-x[0, :]+x[0, :]*x[1, :],
                   2.25-x[0, :]+x[0, :]*x[1, :]*x[1, :],
                   2.625-x[0, :]+x[0, :]*x[1, :]*x[1, :]*x[1, :]))

x0 = np.array((-3.0, -3.0))
x, bl_trace = minimize.least_squares(x0, beale)

# 解を可視化
plot_range = (np.array((-4., 4.)), np.array((-4., 4.)))
minimum = (3., 0.5)
points = np.asarray([t.candidate for t in bl_trace])
plot_2D(beale, 'Beale関数', plot_range, minimum, trace=points)

## ボックス制約

ボックス制約付きの場合の解を見てみましょう：

In [0]:
#@title ボックス制約付きRosenbrock

# 制約を選択
lower = np.array([-.3, -1.])
upper = np.array([0.9, 1.9])
bounds = [lower, upper]

# 複数の初期点を作成し、最小化して軌跡を保存
num_points = 4
px = np.linspace(lower[0]+.1, upper[0]-.1, num_points)
py = np.linspace(lower[1]+.1, upper[1]-.1, num_points)
traces = []
for i in range(num_points):
  for j in range(num_points):
    x0 = np.array((px[j], py[num_points-i-1]))
    _, trace = minimize.least_squares(x0, rosenbrock, bounds, verbose=0)
    traces.append(np.asarray([t.candidate for t in trace]))

# プロット
plot_range = (np.array((-0.5, 1.5)), np.array((-1.5, 2.5)))
minimum = (1, 1)
plot_2D(rosenbrock, 'Rosenbrock関数', plot_range, minimum, traces, bounds)

In [0]:
#@title ボックス制約付きBeale

# 制約を選択
lower = np.array([-2, -1.3])
upper = np.array([1.5, 3.])
bounds = [lower, upper]

# 複数の初期点を作成し、最小化して軌跡を保存
num_points = 5
p0 = np.linspace(lower[0]+.4, upper[0]-.4, num_points)
p1 = np.linspace(lower[1]+.4, upper[1]-.4, num_points)
traces = []

for i in range(num_points):
  for j in range(num_points):
    x0 = np.array((p0[j], p1[i]))
    x, trace = minimize.least_squares(x0, beale, bounds, verbose=0)
    traces.append(np.asarray([t.candidate for t in trace]))

# プロット
plot_range = (np.array((-3., 3.5)), np.array((-3., 4.)))
minimum = (3., 0.5)
plot_2D(beale, 'Beale関数', plot_range, minimum, traces, bounds)

## n次元Rosenbrock

2次元Rosenbrock関数の $n$ 次元への一般化は
$$
f(x) = \sum_{i=1}^{n-1} \left[ 100 \cdot (x_{i+1} - x_{i}^{2})^{2} + \left(1 - x_{i}\right)^{2}\right],
$$
であり、これも最小二乗形式で書くことができ、我々のオプティマイザで解くことができます：

In [0]:
n = 20

def rosenbrock_n(x):
  res0 = [1 - x[i, :] for i in range(n - 1)]
  res1 = [10 * (x[i, :] - x[i + 1, :] ** 2) for i in range(n - 1)]
  return np.asarray(res0 + res1)

x0 = np.zeros(n)
x, _ = minimize.least_squares(x0, rosenbrock_n);

# 期待される解はすべて1のベクトル
assert np.linalg.norm(x-1) < 1e-8

# 逆運動学

順運動学とは、関節角度が与えられた場合に、多関節ボディのデカルト姿勢を計算することです。この計算は容易で明確に定義されています。逆運動学（IK）はその逆を行おうとします：多関節ボディ（通常 **エンドエフェクタ** と呼ばれる）の所望のデカルト姿勢が与えられた場合に、エンドエフェクタをその姿勢にする関節角度を見つけます。IK問題は容易に過剰決定または過少決定になり得ます：
- 過剰決定：エンドエフェクタが所望の姿勢に到達できない、または位置か姿勢のいずれかにはマッチできるが両方にはマッチできない場合。
- 過少決定：多関節チェーンが最低限必要な自由度（DOF）数よりも多くの自由度を持つ場合。ターゲットに到達した後もまだ自由な部分空間が残る（手と肩を固定したまま肘を回転させることを想像してください）。

以下の実装ではこれらの状況に対処していますが、まず現在 **未実装** だが将来追加される可能性のある機能について述べます。
1. 現在は1つのエンドエフェクタとターゲット姿勢のみサポートしています。複数のターゲットを追加するのは簡単です。
2. 現在の実装はキネマティックチェーン内のクォータニオン（ボールジョイントとフリージョイント）をサポートしていません。これを追加することは可能で、`least_squares` 関数への小さな変更（非デカルト接空間の明示的サポート）が必要です。
3. 単純なボックス制約の例のみ示しています（IKの文脈ではジョイント可動域制限を意味します）。衝突回避などの制約を考慮するには、MuJoCoの制約ヤコビアンを利用する必要があります。この例をご覧になりたい場合はお知らせください。

## 7自由度アーム

以下では、MuJoCo MenagerieのFrankaの [Panda](https://github.com/google-deepmind/mujoco_menagerie/tree/main/franka_emika_panda#readme) モデルをコピーしています。変更点は以下の通りです：
1. [scene](https://github.com/google-deepmind/mujoco_menagerie/blob/main/universal_robots_ur5e/scene.xml) の要素を単一のXMLに統合。
2. サイトフレームと重なる3つの色付きの非接触ジオムを持つ「target」mocapボディとサイトを追加。

In [0]:
#@title Panda XML
panda = '''
<mujoco model="panda scene">
  <compiler angle="radian" autolimits="true"/>

  <option integrator="implicitfast"/>

  <statistic center="0.3 0 0.4" extent="1"  meansize=".15"/>

  <visual>
    <headlight diffuse="0.6 0.6 0.6" ambient="0.3 0.3 0.3" specular="0 0 0"/>
    <rgba haze="0.15 0.25 0.35 1"/>
    <global azimuth="120" elevation="-20"/>
  </visual>

  <default>
    <geom condim="1"/>
    <default class="panda">
      <material specular="0.5" shininess="0.25"/>
      <joint armature="0.1" damping="1" axis="0 0 1" range="-2.8973 2.8973"/>
      <general dyntype="none" biastype="affine" ctrlrange="-2.8973 2.8973" forcerange="-87 87"/>
      <default class="finger">
        <joint axis="0 1 0" type="slide" range="0 0.04"/>
      </default>

      <default class="visual">
        <geom type="mesh" contype="0" conaffinity="0" group="2"/>
        <default class="target">
          <geom type="box" size=".015"/>
        </default>
      </default>
      <default class="collision">
        <geom type="mesh" group="3"/>
        <default class="fingertip_pad_collision_1">
          <geom type="box" size="0.0085 0.004 0.0085" pos="0 0.0055 0.0445"/>
        </default>
        <default class="fingertip_pad_collision_2">
          <geom type="box" size="0.003 0.002 0.003" pos="0.0055 0.002 0.05"/>
        </default>
        <default class="fingertip_pad_collision_3">
          <geom type="box" size="0.003 0.002 0.003" pos="-0.0055 0.002 0.05"/>
        </default>
        <default class="fingertip_pad_collision_4">
          <geom type="box" size="0.003 0.002 0.0035" pos="0.0055 0.002 0.0395"/>
        </default>
        <default class="fingertip_pad_collision_5">
          <geom type="box" size="0.003 0.002 0.0035" pos="-0.0055 0.002 0.0395"/>
        </default>
      </default>
    </default>
  </default>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="3072"/>
    <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.2 0.3 0.4" rgb2="0.1 0.2 0.3"
      markrgb="0.8 0.8 0.8" width="300" height="300"/>
    <material name="groundplane" texture="groundplane" texuniform="true" texrepeat="5 5" reflectance="0.2"/>
  </asset>

  <asset>
    <material class="panda" name="white" rgba="1 1 1 1"/>
    <material class="panda" name="off_white" rgba="0.901961 0.921569 0.929412 1"/>
    <material class="panda" name="black" rgba="0.25 0.25 0.25 1"/>
    <material class="panda" name="green" rgba="0 1 0 1"/>
    <material class="panda" name="light_blue" rgba="0.039216 0.541176 0.780392 1"/>

    <!-- Collision meshes -->
    <mesh name="link0_c" file="link0.stl"/>
    <mesh name="link1_c" file="link1.stl"/>
    <mesh name="link2_c" file="link2.stl"/>
    <mesh name="link3_c" file="link3.stl"/>
    <mesh name="link4_c" file="link4.stl"/>
    <mesh name="link5_c0" file="link5_collision_0.obj"/>
    <mesh name="link5_c1" file="link5_collision_1.obj"/>
    <mesh name="link5_c2" file="link5_collision_2.obj"/>
    <mesh name="link6_c" file="link6.stl"/>
    <mesh name="link7_c" file="link7.stl"/>
    <mesh name="hand_c" file="hand.stl"/>

    <!-- Visual meshes -->
    <mesh file="link0_0.obj"/>
    <mesh file="link0_1.obj"/>
    <mesh file="link0_2.obj"/>
    <mesh file="link0_3.obj"/>
    <mesh file="link0_4.obj"/>
    <mesh file="link0_5.obj"/>
    <mesh file="link0_7.obj"/>
    <mesh file="link0_8.obj"/>
    <mesh file="link0_9.obj"/>
    <mesh file="link0_10.obj"/>
    <mesh file="link0_11.obj"/>
    <mesh file="link1.obj"/>
    <mesh file="link2.obj"/>
    <mesh file="link3_0.obj"/>
    <mesh file="link3_1.obj"/>
    <mesh file="link3_2.obj"/>
    <mesh file="link3_3.obj"/>
    <mesh file="link4_0.obj"/>
    <mesh file="link4_1.obj"/>
    <mesh file="link4_2.obj"/>
    <mesh file="link4_3.obj"/>
    <mesh file="link5_0.obj"/>
    <mesh file="link5_1.obj"/>
    <mesh file="link5_2.obj"/>
    <mesh file="link6_0.obj"/>
    <mesh file="link6_1.obj"/>
    <mesh file="link6_2.obj"/>
    <mesh file="link6_3.obj"/>
    <mesh file="link6_4.obj"/>
    <mesh file="link6_5.obj"/>
    <mesh file="link6_6.obj"/>
    <mesh file="link6_7.obj"/>
    <mesh file="link6_8.obj"/>
    <mesh file="link6_9.obj"/>
    <mesh file="link6_10.obj"/>
    <mesh file="link6_11.obj"/>
    <mesh file="link6_12.obj"/>
    <mesh file="link6_13.obj"/>
    <mesh file="link6_14.obj"/>
    <mesh file="link6_15.obj"/>
    <mesh file="link6_16.obj"/>
    <mesh file="link7_0.obj"/>
    <mesh file="link7_1.obj"/>
    <mesh file="link7_2.obj"/>
    <mesh file="link7_3.obj"/>
    <mesh file="link7_4.obj"/>
    <mesh file="link7_5.obj"/>
    <mesh file="link7_6.obj"/>
    <mesh file="link7_7.obj"/>
    <mesh file="hand_0.obj"/>
    <mesh file="hand_1.obj"/>
    <mesh file="hand_2.obj"/>
    <mesh file="hand_3.obj"/>
    <mesh file="hand_4.obj"/>
    <mesh file="finger_0.obj"/>
    <mesh file="finger_1.obj"/>
  </asset>

  <worldbody>
    <light pos="0 0 1.5" dir="0 0 -1" directional="true"/>
    <geom name="floor" size="0 0 0.05" type="plane" material="groundplane"/>

    <light name="top" pos="0 0 2" mode="trackcom"/>
    <body name="link0" childclass="panda">
      <inertial mass="0.629769" pos="-0.041018 -0.00014 0.049974"
        fullinertia="0.00315 0.00388 0.004285 8.2904e-7 0.00015 8.2299e-6"/>
      <geom mesh="link0_0" material="off_white" class="visual"/>
      <geom mesh="link0_1" material="black" class="visual"/>
      <geom mesh="link0_2" material="off_white" class="visual"/>
      <geom mesh="link0_3" material="black" class="visual"/>
      <geom mesh="link0_4" material="off_white" class="visual"/>
      <geom mesh="link0_5" material="black" class="visual"/>
      <geom mesh="link0_7" material="white" class="visual"/>
      <geom mesh="link0_8" material="white" class="visual"/>
      <geom mesh="link0_9" material="black" class="visual"/>
      <geom mesh="link0_10" material="off_white" class="visual"/>
      <geom mesh="link0_11" material="white" class="visual"/>
      <geom mesh="link0_c" class="collision"/>
      <body name="link1" pos="0 0 0.333">
        <inertial mass="4.970684" pos="0.003875 0.002081 -0.04762"
          fullinertia="0.70337 0.70661 0.0091170 -0.00013900 0.0067720 0.019169"/>
        <joint name="joint1"/>
        <geom material="white" mesh="link1" class="visual"/>
        <geom mesh="link1_c" class="collision"/>
        <body name="link2" quat="1 -1 0 0">
          <inertial mass="0.646926" pos="-0.003141 -0.02872 0.003495"
            fullinertia="0.0079620 2.8110e-2 2.5995e-2 -3.925e-3 1.0254e-2 7.04e-4"/>
          <joint name="joint2" range="-1.7628 1.7628"/>
          <geom material="white" mesh="link2" class="visual"/>
          <geom mesh="link2_c" class="collision"/>
          <body name="link3" pos="0 -0.316 0" quat="1 1 0 0">
            <joint name="joint3"/>
            <inertial mass="3.228604" pos="2.7518e-2 3.9252e-2 -6.6502e-2"
              fullinertia="3.7242e-2 3.6155e-2 1.083e-2 -4.761e-3 -1.1396e-2 -1.2805e-2"/>
            <geom mesh="link3_0" material="white" class="visual"/>
            <geom mesh="link3_1" material="white" class="visual"/>
            <geom mesh="link3_2" material="white" class="visual"/>
            <geom mesh="link3_3" material="black" class="visual"/>
            <geom mesh="link3_c" class="collision"/>
            <body name="link4" pos="0.0825 0 0" quat="1 1 0 0">
              <inertial mass="3.587895" pos="-5.317e-2 1.04419e-1 2.7454e-2"
                fullinertia="2.5853e-2 1.9552e-2 2.8323e-2 7.796e-3 -1.332e-3 8.641e-3"/>
              <joint name="joint4" range="-3.0718 -0.0698"/>
              <geom mesh="link4_0" material="white" class="visual"/>
              <geom mesh="link4_1" material="white" class="visual"/>
              <geom mesh="link4_2" material="black" class="visual"/>
              <geom mesh="link4_3" material="white" class="visual"/>
              <geom mesh="link4_c" class="collision"/>
              <body name="link5" pos="-0.0825 0.384 0" quat="1 -1 0 0">
                <inertial mass="1.225946" pos="-1.1953e-2 4.1065e-2 -3.8437e-2"
                  fullinertia="3.5549e-2 2.9474e-2 8.627e-3 -2.117e-3 -4.037e-3 2.29e-4"/>
                <joint name="joint5"/>
                <geom mesh="link5_0" material="black" class="visual"/>
                <geom mesh="link5_1" material="white" class="visual"/>
                <geom mesh="link5_2" material="white" class="visual"/>
                <geom mesh="link5_c0" class="collision"/>
                <geom mesh="link5_c1" class="collision"/>
                <geom mesh="link5_c2" class="collision"/>
                <body name="link6" quat="1 1 0 0">
                  <inertial mass="1.666555" pos="6.0149e-2 -1.4117e-2 -1.0517e-2"
                    fullinertia="1.964e-3 4.354e-3 5.433e-3 1.09e-4 -1.158e-3 3.41e-4"/>
                  <joint name="joint6" range="-0.0175 3.7525"/>
                  <geom mesh="link6_0" material="off_white" class="visual"/>
                  <geom mesh="link6_1" material="white" class="visual"/>
                  <geom mesh="link6_2" material="black" class="visual"/>
                  <geom mesh="link6_3" material="white" class="visual"/>
                  <geom mesh="link6_4" material="white" class="visual"/>
                  <geom mesh="link6_5" material="white" class="visual"/>
                  <geom mesh="link6_6" material="white" class="visual"/>
                  <geom mesh="link6_7" material="light_blue" class="visual"/>
                  <geom mesh="link6_8" material="light_blue" class="visual"/>
                  <geom mesh="link6_9" material="black" class="visual"/>
                  <geom mesh="link6_10" material="black" class="visual"/>
                  <geom mesh="link6_11" material="white" class="visual"/>
                  <geom mesh="link6_12" material="green" class="visual"/>
                  <geom mesh="link6_13" material="white" class="visual"/>
                  <geom mesh="link6_14" material="black" class="visual"/>
                  <geom mesh="link6_15" material="black" class="visual"/>
                  <geom mesh="link6_16" material="white" class="visual"/>
                  <geom mesh="link6_c" class="collision"/>
                  <body name="link7" pos="0.088 0 0" quat="1 1 0 0">
                    <inertial mass="7.35522e-01" pos="1.0517e-2 -4.252e-3 6.1597e-2"
                      fullinertia="1.2516e-2 1.0027e-2 4.815e-3 -4.28e-4 -1.196e-3 -7.41e-4"/>
                    <joint name="joint7"/>
                    <geom mesh="link7_0" material="white" class="visual"/>
                    <geom mesh="link7_1" material="black" class="visual"/>
                    <geom mesh="link7_2" material="black" class="visual"/>
                    <geom mesh="link7_3" material="black" class="visual"/>
                    <geom mesh="link7_4" material="black" class="visual"/>
                    <geom mesh="link7_5" material="black" class="visual"/>
                    <geom mesh="link7_6" material="black" class="visual"/>
                    <geom mesh="link7_7" material="white" class="visual"/>
                    <geom mesh="link7_c" class="collision"/>
                    <body name="hand" pos="0 0 0.107" quat="0.9238795 0 0 -0.3826834">
                      <inertial mass="0.73" pos="-0.01 0 0.03" diaginertia="0.001 0.0025 0.0017"/>
                      <geom mesh="hand_0" material="off_white" class="visual"/>
                      <geom mesh="hand_1" material="black" class="visual"/>
                      <geom mesh="hand_2" material="black" class="visual"/>
                      <geom mesh="hand_3" material="white" class="visual"/>
                      <geom mesh="hand_4" material="off_white" class="visual"/>
                      <geom mesh="hand_c" class="collision"/>
                      <site name="effector" pos="0 0 0.08"/>
                    </body>
                  </body>
                </body>
              </body>
            </body>
          </body>
        </body>
      </body>
    </body>

    <body name="target" pos="0 .6 .4" quat=".5 -.2 1 0" mocap="true">
      <site name="target" group="2"/>
      <geom class="target" fromto=".07 0 0 .15 0 0" rgba="1 0 0 1"/>
      <geom class="target" fromto="0 .07 0 0 .15 0" rgba="0 1 0 1"/>
      <geom class="target" fromto="0 0 .07 0 0 .15" rgba="0 0 1 1"/>
    </body>
  </worldbody>

  <actuator>
    <general class="panda" name="actuator1" joint="joint1" gainprm="4500" biasprm="0 -4500 -450"/>
    <general class="panda" name="actuator2" joint="joint2" gainprm="4500" biasprm="0 -4500 -450"
      ctrlrange="-1.7628 1.7628"/>
    <general class="panda" name="actuator3" joint="joint3" gainprm="3500" biasprm="0 -3500 -350"/>
    <general class="panda" name="actuator4" joint="joint4" gainprm="3500" biasprm="0 -3500 -350"
      ctrlrange="-3.0718 -0.0698"/>
    <general class="panda" name="actuator5" joint="joint5" gainprm="2000" biasprm="0 -2000 -200" forcerange="-12 12"/>
    <general class="panda" name="actuator6" joint="joint6" gainprm="2000" biasprm="0 -2000 -200" forcerange="-12 12"
      ctrlrange="-0.0175 3.7525"/>
    <general class="panda" name="actuator7" joint="joint7" gainprm="2000" biasprm="0 -2000 -200" forcerange="-12 12"/>
  </actuator>

  <keyframe>
    <key name="home" qpos="0 0 0 -1.57079 0 1.57079 -0.7853" ctrl="0 0 0 -1.57079 0 1.57079 -0.7853"/>
  </keyframe>
</mujoco>
'''

In [0]:
#@title Pandaアセットの取得
!git clone https://github.com/google-deepmind/mujoco_menagerie
from os import walk
from os.path import join
assets_path = 'mujoco_menagerie/franka_emika_panda/assets/'
asset_names = next(walk(assets_path), (None, None, []))[2]
assets = {}
for name in asset_names:
  with open(join(assets_path, name), 'rb') as f:
    assets[name] = f.read()


In [0]:
#@title モデルの読み込みとレンダリング {vertical-output: true}
model = mujoco.MjModel.from_xml_string(panda, assets)
data = mujoco.MjData(model)

# 状態を「home」キーフレームにリセット
key = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_KEY, 'home')
mujoco.mj_resetDataKeyframe(model, data, key)
mujoco.mj_forward(model, data)

# レンダラーが存在する場合は閉じる
if 'renderer' in locals():
  renderer.close()

# レンダラーとカメラの作成
renderer = mujoco.Renderer(model, height=360, width=480)
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 1.7
camera.elevation = -15
camera.azimuth = -130
camera.lookat = (0, 0, .3)

# サイトフレームとラベルを表示
voption = mujoco.MjvOption()
voption.frame = mujoco.mjtFrame.mjFRAME_SITE
voption.label = mujoco.mjtLabel.mjLABEL_SITE
renderer.update_scene(data, camera, voption)
voption.label = mujoco.mjtLabel.mjLABEL_NONE

media.show_image(renderer.render())

ターゲットサイトフレームには軸に沿ったボックスが付いており、エンドエフェクタフレームと視覚的に区別できるようになっています。

## 姿勢の重み付け

IK問題の一般的な特徴として、姿勢のマッチングには位置と向きの両方をマッチさせる必要があります。しかし、これらの残差は異なる単位を持ちます。位置の差は長さの単位（例えばメートル）であり、向きの差（クォータニオン差の対数写像）は無次元のラジアンです。ユーザーは、向きのマッチングと位置のマッチングをどの程度「重視する」かを示す変換係数を提供する必要があります。この係数は長さの単位を持つため、`radius` と呼んでいます。これはエフェクタフレーム周りの球の半径と考えることができ、その球面上で角度誤差が測定されます。

## IKの正則化

Pandaアームは7自由度（DOF）を持つため、IK問題は6つの制約（並進3 + 回転3）に対して過少決定です。一意の解を得るために、構成をあるベース姿勢に引き寄せる固定の正則化項を導入します。

## 問題定義

これで問題定義を見る準備が整いました：

### 制約と初期推定値

In [0]:
# ジョイント限界での制約
bounds = [model.jnt_range[:, 0], model.jnt_range[:, 1]]

# 初期推定値は「home」キーフレーム
x0 = model.key('home').qpos

### IK残差

In [0]:
def ik(x, pos=None, quat=None, radius=0.04, reg=1e-3, reg_target=None):
  """逆運動学の残差。

  引数:
    x: 関節角度。
    pos: エンドエフェクタの目標位置。
    quat: エンドエフェクタの目標姿勢。
    radius: 3Dクロスのスケーリング。

  戻り値:
    逆運動学タスクの残差。
  """

  # mocapボディをターゲットに移動
  id = model.body('target').mocapid
  data.mocap_pos[id] = model.body('target').pos if pos is None else pos
  data.mocap_quat[id] = model.body('target').quat if quat is None else quat

  # qposを設定し、順運動学を計算
  res = []
  for i in range(x.shape[1]):
    data.qpos = x[:, i]
    mujoco.mj_kinematics(model, data)

    # 位置の残差
    res_pos = data.site('effector').xpos - data.site('target').xpos

    # エフェクタのクォータニオン、mju_mat2quatを使用
    effector_quat = np.empty(4)
    mujoco.mju_mat2Quat(effector_quat, data.site('effector').xmat)

    # ターゲットのクォータニオン、サイトがボディに整列していることを利用
    target_quat = data.body('target').xquat

    # 姿勢の残差：クォータニオンの差
    res_quat = np.empty(3)
    mujoco.mju_subQuat(res_quat, target_quat, effector_quat)
    res_quat *= radius

    # 正則化の残差
    reg_target = model.key('home').qpos if reg_target is None else reg_target
    res_reg = reg * (x[:, i] - reg_target)

    res_i = np.hstack((res_pos, res_quat, res_reg))
    res.append(np.atleast_2d(res_i).T)

  return np.hstack(res)

### 解析的ヤコビアン

IK問題はもう一つの点で特殊です。一般的なケースとは異なり、ここではMuJoCoによって解析的ヤコビアンを計算できます。このヤコビアンを書き下し、有限差分を使用した場合の解法時間と解析的ヤコビアンを使用した場合の解法時間を比較してみましょう。

In [0]:
def ik_jac(x, res, pos=None, quat=None, radius=.04, reg=1e-3):
  """逆運動学残差の解析的ヤコビアン

  引数:
    x: 関節角度。
    pos: エンドエフェクタの目標位置。
    quat: エンドエフェクタの目標姿勢。
    radius: 3Dクロスのスケーリング。

  戻り値:
    逆運動学タスクのヤコビアン。
  """
  # least_squares()はxにおける残差の値を渡しますが、
  # ここでは使用しません。
  del res

  # mj_kinematicsとmj_comPosを呼び出す（ヤコビアンに必要）
  mujoco.mj_kinematics(model, data)
  mujoco.mj_comPos(model, data)

  # エンドエフェクタサイトのヤコビアンを取得
  jac_pos = np.empty((3, model.nv))
  jac_quat = np.empty((3, model.nv))
  mujoco.mj_jacSite(model, data, jac_pos, jac_quat, data.site('effector').id)

  # Deffector（3x3のmju_subquatヤコビアン）を取得
  effector_quat = np.empty(4)
  mujoco.mju_mat2Quat(effector_quat, data.site('effector').xmat)
  target_quat = data.body('target').xquat
  Deffector = np.empty((3, 3))
  mujoco.mjd_subQuat(target_quat, effector_quat, None, Deffector)

  # ターゲットフレームに回転し、subQuatヤコビアンを乗算し、radiusでスケーリング
  target_mat = data.site('target').xmat.reshape(3, 3)
  mat = radius * Deffector.T @ target_mat.T
  jac_quat = mat @ jac_quat

  # 正則化のヤコビアン
  jac_reg = reg * np.eye(model.nv)

  return np.vstack((jac_pos, jac_quat, jac_reg))

有限差分ヤコビアンと解析的ヤコビアンの性能を比較してみましょう：

In [0]:
print('有限差分ヤコビアン：')
x_fd, _ = minimize.least_squares(x0, ik, bounds, verbose=1);
print('解析的ヤコビアン：')
x_analytic, _ = minimize.least_squares(x0, ik, bounds, jacobian=ik_jac,
                                       verbose=1, check_derivatives=True);

# ほぼ同一の解が得られたことを確認
assert np.linalg.norm(x_fd - x_analytic) < 1e-5

素晴らしい高速化です！IK問題が難しくなるほど、この差はさらに顕著になります。数セル先でより包括的なタイミング比較を行います（この特定の構成はかなり遅く解かれます）。

`check_derivatives=True` を渡して、解析的ヤコビアンが正しいことを検証するよう関数に依頼したことに注目してください。最初のタイムステップで内部の有限差分関数と比較することで検証が行われます。

## 解の可視化

問題の解がどのように見えるか確認しましょう。

In [0]:
#@title 基本的な解 {vertical-output: true}

x, _ = minimize.least_squares(x0, ik, bounds, jacobian=ik_jac,
                              verbose=0);

# 更新と可視化
data.qpos = x
mujoco.mj_kinematics(model, data)
mujoco.mj_camlight(model, data)
camera.distance = 1
camera.lookat = data.site('effector').xpos
renderer.update_scene(data, camera, voption)
media.show_image(renderer.render())

`radius` 引数の効果をより良く理解するために、IKソルバーにより難しいターゲット姿勢を与え、radiusを変化させてみましょう：

In [0]:
#@title 小さな `radius` {vertical-output: true}

pos = np.array((.2, .8, .5))
quat = np.array((1, 1, 0, 0))
radius = 0.04

ik_target = lambda x: ik(x, pos=pos, quat=quat, radius=radius)
jac_target = lambda x, r: ik_jac(x, r, pos=pos, quat=quat,
                                 radius=radius)

x, _ = minimize.least_squares(x0, ik_target, bounds,
                              jacobian=jac_target,
                              verbose=0);

# 更新と可視化
data.qpos = x
mujoco.mj_kinematics(model, data)
mujoco.mj_camlight(model, data)
camera.distance = 1
camera.lookat = data.site('effector').xpos
camera.azimuth = -150
camera.elevation = -20
renderer.update_scene(data, camera, voption)
media.show_image(renderer.render())

位置はよくマッチしていますが、向きは大きくずれていることがわかります。

In [0]:
#@title 大きな `radius` {vertical-output: true}

pos = np.array((.2, .8, .5))
quat = np.array((1, 1, 0, 0))
radius = 0.5

ik_target = lambda x: ik(x, pos=pos, quat=quat, radius=radius)
jac_target = lambda x, r: ik_jac(x, r, pos=pos, quat=quat,
                                 radius=radius)

x, _ = minimize.least_squares(x0, ik_target, bounds,
                              jacobian=jac_target,
                              verbose=0);

# 更新と可視化
data.qpos = x
mujoco.mj_kinematics(model, data)
mujoco.mj_camlight(model, data)
camera.distance = 1
camera.lookat = data.site('effector').xpos
camera.azimuth = -150
camera.elevation = -20
renderer.update_scene(data, camera, voption)
media.show_image(renderer.render())

今度は向きがよりよくマッチしていますが、位置誤差がはるかに大きくなっています。

## 滑らかな動き

エンドエフェクタの軌道を滑らかに追跡したい場合がよくあります。滑らかなターゲット軌道を作成してみましょう：

In [0]:
# @title 連続的なターゲット軌道 {vertical-output: true}
def pose(time):
  pos = (0.4 * np.sin(time),
         0.4 * np.cos(time),
         0.4 + 0.2 * np.sin(3 * time))
  quat = np.array((1.0, np.sin(2 * time), np.sin(time), 0))
  quat /= np.linalg.norm(quat)
  return pos, quat

# フレーム数
n_frame = 400

# カメラをリセットし、アームを真上に向ける
camera.distance = 1.5
camera.elevation = -15
camera.azimuth = -130
camera.lookat = (0, 0, 0.3)
mujoco.mj_resetData(model, data)

frames = []
for t in np.linspace(0, 2 * np.pi, n_frame):
  # ターゲットを移動
  pos, quat = pose(t)
  id = model.body('target').mocapid
  data.mocap_pos[id] = pos
  data.mocap_quat[id] = quat

  mujoco.mj_kinematics(model, data)
  mujoco.mj_camlight(model, data)
  renderer.update_scene(data, camera, voption)

  frames.append(renderer.render())

media.show_video(frames)

各フレームに対してIK問題を独立に解き、常に初期推定値 `x0` から開始した場合のアームの動きを見てみましょう。これを行いながら、解析的導関数の有無による解法のタイミングを計測して、最適化にかかる時間をより良く把握します。

In [0]:
#@title 毎フレームの完全なIK {vertical-output: true}

frames = []
time_fd = []
time_analytic = []
for t in np.linspace(0, 2 * np.pi, n_frame):
  # ターゲット姿勢を取得
  pos, quat = pose(t)

  # IK問題を定義
  ik_target = lambda x: ik(x, pos=pos, quat=quat)
  jac_target = lambda x, r: ik_jac(x, r, pos=pos, quat=quat)

  # タイミングを計測しながら解法、有限差分ヤコビアン
  t_start = time.time()
  x, _ = minimize.least_squares(x0, ik_target, bounds,
                                verbose=0);
  time_fd.append(1000*(time.time() - t_start))

  # タイミングを計測しながら解法、解析的ヤコビアン
  t_start = time.time()
  x, _ = minimize.least_squares(x0, ik_target, bounds,
                                jacobian=jac_target,
                                verbose=0);
  time_analytic.append(1000*(time.time() - t_start))

  mujoco.mj_kinematics(model, data)
  mujoco.mj_camlight(model, data)
  renderer.update_scene(data, camera, voption)

  frames.append(renderer.render())

media.show_video(frames, loop=False)

mean_analytic = np.asarray(time_analytic).mean()
mean_fd = np.asarray(time_fd).mean()
prfx = '平均解法時間（'
print(prfx + f'有限差分ヤコビアン）: {mean_fd:3.1f}ms')
print(prfx + f'解析的ヤコビアン）: {mean_analytic:3.1f}ms')

解は見つかっていますが、あちこちで「グリッチ」が発生しています！IK問題にはしばしば複数の局所最小値があり、毎回ゼロから解いているため、やや恣意的に異なる局所最小値に到達してしまいます。

これを軽減する非常に簡単な方法は、ソルバーを前回の解で初期化（「ウォームスタート」）することです：

In [0]:
#@title 前回の解でウォームスタート {vertical-output: true}

frames = []
x = x0
for t in np.linspace(0, 2 * np.pi, n_frame):
  # ターゲット姿勢を取得
  pos, quat = pose(t)

  # IK問題を定義
  ik_target = lambda x: ik(x, pos=pos, quat=quat)
  jac_target = lambda x, r: ik_jac(x, r, pos=pos, quat=quat)

  x, _ = minimize.least_squares(x, ik_target, bounds,
                                jacobian=jac_target,
                                verbose=0);

  mujoco.mj_kinematics(model, data)
  mujoco.mj_camlight(model, data)
  renderer.update_scene(data, camera, voption)

  frames.append(renderer.render())

media.show_video(frames, loop=False)

これは *はるかに* 良くなりましたが、まだいくつかの「グリッチ」が残っています。これを軽減する簡単な方法は、正則化のターゲットも前回の解にすることです。実質的に、ソルバーにあまり遠くに行かないよう伝えていることになります。

In [0]:
#@title 前回の解でウォームスタートし正則化 {vertical-output: true}

frames = []
x = x0
for t in np.linspace(0, 2 * np.pi, n_frame):
  # ターゲット姿勢を取得
  pos, quat = pose(t)

  x_prev = x.copy()

  # IK問題を定義
  ik_target = lambda x: ik(x, pos=pos, quat=quat,
                           reg_target=x_prev, reg=.1)
  jac_target = lambda x, r: ik_jac(x, r, pos=pos, quat=quat)

  x, _ = minimize.least_squares(x, ik_target, bounds,
                                jacobian=jac_target,
                                verbose=0);

  mujoco.mj_kinematics(model, data)
  mujoco.mj_camlight(model, data)
  renderer.update_scene(data, camera, voption)

  frames.append(renderer.render())

media.show_video(frames, loop=False)

# 非2次ノルム

ここからは `least_squares` を使用して軌道最適化（制御）問題を解きます。これは本来の用途ではありませんが、この関数の汎用性と強力さを示しています。

通常の最小二乗法を使用した後、カスタムの **非2次ノルム** を定義して再び解きます。

以下では、MuJoCoの [標準ヒューマノイドモデル](https://github.com/google-deepmind/mujoco/blob/main/model/humanoid/humanoid.xml) をコピーしています。変更点は以下の通りです：
1. ピンク色の球体サイトを持つ「target」mocapボディを追加。
2. 右手の色をピンクに変更。
3. トルクアクチュエータを位置アクチュエータに置換。
4. 残差に対応するセンサーを追加。

In [0]:
#@title Humanoid XML
xml = """
<mujoco model="Humanoid">
  <option timestep="0.005"/>

  <visual>
    <map force="0.1" zfar="30"/>
    <rgba haze="0.15 0.25 0.35 1"/>
    <global offwidth="2560" offheight="1440" elevation="-20" azimuth="120"/>
  </visual>

  <statistic center="0 0 0.7"/>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="body" type="cube" builtin="flat" mark="cross" width="128" height="128" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" markrgb="1 1 1" random="0.01"/>
    <material name="body" texture="body" texuniform="true" rgba="0.8 0.6 .4 1"/>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="1 1" texuniform="true" reflectance=".2"/>
  </asset>

  <default>
    <position inheritrange="0.95"/>
    <default class="body">

      <!-- geoms -->
      <geom type="capsule" condim="1" friction=".7" solimp="0.9 .99 .003" solref=".015 1" material="body" group="1"/>
      <default class="thigh">
        <geom size=".06"/>
      </default>
      <default class="shin">
        <geom fromto="0 0 0 0 0 -.3"  size=".049"/>
      </default>
      <default class="foot">
        <geom size=".027"/>
        <default class="foot1">
          <geom fromto="-.07 -.01 0 .14 -.03 0"/>
        </default>
        <default class="foot2">
          <geom fromto="-.07 .01 0 .14  .03 0"/>
        </default>
      </default>
      <default class="arm_upper">
        <geom size=".04"/>
      </default>
      <default class="arm_lower">
        <geom size=".031"/>
      </default>
      <default class="hand">
        <geom type="sphere" size=".04"/>
      </default>

      <!-- joints -->
      <joint type="hinge" damping=".2" stiffness="1" armature=".01" limited="true" solimplimit="0 .99 .01"/>
      <default class="joint_big">
        <joint damping="5" stiffness="10"/>
        <default class="hip_x">
          <joint range="-30 10"/>
        </default>
        <default class="hip_z">
          <joint range="-60 35"/>
        </default>
        <default class="hip_y">
          <joint axis="0 1 0" range="-150 20"/>
        </default>
        <default class="joint_big_stiff">
          <joint stiffness="20"/>
        </default>
      </default>
      <default class="knee">
        <joint pos="0 0 .02" axis="0 -1 0" range="-160 2"/>
      </default>
      <default class="ankle">
        <joint range="-50 50"/>
        <default class="ankle_y">
          <joint pos="0 0 .08" axis="0 1 0" stiffness="6"/>
        </default>
        <default class="ankle_x">
          <joint pos="0 0 .04" stiffness="3"/>
        </default>
      </default>
      <default class="shoulder">
        <joint range="-85 60"/>
      </default>
      <default class="elbow">
        <joint range="-100 50" stiffness="0"/>
      </default>
    </default>
  </default>

  <worldbody>
    <body name="target" pos=".2 -.2 1" mocap="true">
      <site name="target" size=".05" rgba="1 0 1 .4"/>
    </body>
    <geom name="floor" size="0 0 .05" type="plane" material="grid" condim="3"/>
    <light name="spotlight" mode="targetbodycom" target="torso" diffuse=".8 .8 .8" specular="0.3 0.3 0.3" pos="0 -6 4" cutoff="30"/>
    <body name="torso" pos="0 0 1.282" childclass="body">
      <light name="top" pos="0 0 2" mode="trackcom"/>
      <camera name="back" pos="-3 0 1" xyaxes="0 -1 0 1 0 2" mode="trackcom"/>
      <camera name="side" pos="0 -3 1" xyaxes="1 0 0 0 1 2" mode="trackcom"/>
      <freejoint name="root"/>
      <geom name="torso" fromto="0 -.07 0 0 .07 0" size=".07"/>
      <geom name="waist_upper" fromto="-.01 -.06 -.12 -.01 .06 -.12" size=".06"/>
      <body name="head" pos="0 0 .19">
        <geom name="head" type="sphere" size=".09"/>
        <camera name="egocentric" pos=".09 0 0" xyaxes="0 -1 0 .1 0 1" fovy="80"/>
      </body>
      <body name="waist_lower" pos="-.01 0 -.26">
        <geom name="waist_lower" fromto="0 -.06 0 0 .06 0" size=".06"/>
        <joint name="abdomen_z" pos="0 0 .065" axis="0 0 1" range="-45 45" class="joint_big_stiff"/>
        <joint name="abdomen_y" pos="0 0 .065" axis="0 1 0" range="-75 30" class="joint_big"/>
        <body name="pelvis" pos="0 0 -.165">
          <joint name="abdomen_x" pos="0 0 .1" axis="1 0 0" range="-35 35" class="joint_big"/>
          <geom name="butt" fromto="-.02 -.07 0 -.02 .07 0" size=".09"/>
          <body name="thigh_right" pos="0 -.1 -.04">
            <joint name="hip_x_right" axis="1 0 0" class="hip_x"/>
            <joint name="hip_z_right" axis="0 0 1" class="hip_z"/>
            <joint name="hip_y_right" class="hip_y"/>
            <geom name="thigh_right" fromto="0 0 0 0 .01 -.34" class="thigh"/>
            <body name="shin_right" pos="0 .01 -.4">
              <joint name="knee_right" class="knee"/>
              <geom name="shin_right" class="shin"/>
              <body name="foot_right" pos="0 0 -.39">
                <joint name="ankle_y_right" class="ankle_y"/>
                <joint name="ankle_x_right" class="ankle_x" axis="1 0 .5"/>
                <geom name="foot1_right" class="foot1"/>
                <geom name="foot2_right" class="foot2"/>
              </body>
            </body>
          </body>
          <body name="thigh_left" pos="0 .1 -.04">
            <joint name="hip_x_left" axis="-1 0 0" class="hip_x"/>
            <joint name="hip_z_left" axis="0 0 -1" class="hip_z"/>
            <joint name="hip_y_left" class="hip_y"/>
            <geom name="thigh_left" fromto="0 0 0 0 -.01 -.34" class="thigh"/>
            <body name="shin_left" pos="0 -.01 -.4">
              <joint name="knee_left" class="knee"/>
              <geom name="shin_left" fromto="0 0 0 0 0 -.3" class="shin"/>
              <body name="foot_left" pos="0 0 -.39">
                <joint name="ankle_y_left" class="ankle_y"/>
                <joint name="ankle_x_left" class="ankle_x" axis="-1 0 -.5"/>
                <geom name="foot1_left" class="foot1"/>
                <geom name="foot2_left" class="foot2"/>
              </body>
            </body>
          </body>
        </body>
      </body>
      <body name="upper_arm_right" pos="0 -.17 .06">
        <joint name="shoulder1_right" axis="2 1 1"  class="shoulder"/>
        <joint name="shoulder2_right" axis="0 -1 1" class="shoulder"/>
        <geom name="upper_arm_right" fromto="0 0 0 .16 -.16 -.16" class="arm_upper"/>
        <body name="lower_arm_right" pos=".18 -.18 -.18">
          <joint name="elbow_right" axis="0 -1 1" class="elbow"/>
          <geom name="lower_arm_right" fromto=".01 .01 .01 .17 .17 .17" class="arm_lower"/>
          <body name="hand_right" pos=".18 .18 .18">
            <geom name="hand_right" zaxis="1 1 1" class="hand" rgba="1 0 1 1"/>
          </body>
        </body>
      </body>
      <body name="upper_arm_left" pos="0 .17 .06">
        <joint name="shoulder1_left" axis="-2 1 -1" class="shoulder"/>
        <joint name="shoulder2_left" axis="0 -1 -1"  class="shoulder"/>
        <geom name="upper_arm_left" fromto="0 0 0 .16 .16 -.16" class="arm_upper"/>
        <body name="lower_arm_left" pos=".18 .18 -.18">
          <joint name="elbow_left" axis="0 -1 -1" class="elbow"/>
          <geom name="lower_arm_left" fromto=".01 -.01 .01 .17 -.17 .17" class="arm_lower"/>
          <body name="hand_left" pos=".18 -.18 .18">
            <geom name="hand_left" zaxis="1 -1 1" class="hand"/>
          </body>
        </body>
      </body>
    </body>
  </worldbody>

  <contact>
    <exclude body1="waist_lower" body2="thigh_right"/>
    <exclude body1="waist_lower" body2="thigh_left"/>
  </contact>

  <tendon>
    <fixed name="hamstring_right" limited="true" range="-0.3 2">
      <joint joint="hip_y_right" coef=".5"/>
      <joint joint="knee_right" coef="-.5"/>
    </fixed>
    <fixed name="hamstring_left" limited="true" range="-0.3 2">
      <joint joint="hip_y_left" coef=".5"/>
      <joint joint="knee_left" coef="-.5"/>
    </fixed>
  </tendon>

  <actuator>
    <position name="abdomen_z"       kp="40"  joint="abdomen_z"/>
    <position name="abdomen_y"       kp="40"  joint="abdomen_y"/>
    <position name="abdomen_x"       kp="40"  joint="abdomen_x"/>
    <position name="hip_x_right"     kp="40"  joint="hip_x_right"/>
    <position name="hip_z_right"     kp="40"  joint="hip_z_right"/>
    <position name="hip_y_right"     kp="120" joint="hip_y_right"/>
    <position name="knee_right"      kp="80"  joint="knee_right"/>
    <position name="ankle_y_right"   kp="20"  joint="ankle_y_right"/>
    <position name="ankle_x_right"   kp="20"  joint="ankle_x_right"/>
    <position name="hip_x_left"      kp="40"  joint="hip_x_left"/>
    <position name="hip_z_left"      kp="40"  joint="hip_z_left"/>
    <position name="hip_y_left"      kp="120" joint="hip_y_left"/>
    <position name="knee_left"       kp="80"  joint="knee_left"/>
    <position name="ankle_y_left"    kp="20"  joint="ankle_y_left"/>
    <position name="ankle_x_left"    kp="20"  joint="ankle_x_left"/>
    <position name="shoulder1_right" kp="20"  joint="shoulder1_right"/>
    <position name="shoulder2_right" kp="20"  joint="shoulder2_right"/>
    <position name="elbow_right"     kp="40"  joint="elbow_right"/>
    <position name="shoulder1_left"  kp="20"  joint="shoulder1_left"/>
    <position name="shoulder2_left"  kp="20"  joint="shoulder2_left"/>
    <position name="elbow_left"      kp="40"  joint="elbow_left"/>
  </actuator>

  <sensor>
    <framepos objtype="geom" objname="hand_right" reftype="xbody" refname="target"/>
    <actuatorfrc actuator="abdomen_z"/>
    <actuatorfrc actuator="abdomen_y"/>
    <actuatorfrc actuator="abdomen_x"/>
    <actuatorfrc actuator="hip_x_right"/>
    <actuatorfrc actuator="hip_z_right"/>
    <actuatorfrc actuator="hip_y_right"/>
    <actuatorfrc actuator="knee_right"/>
    <actuatorfrc actuator="ankle_y_right"/>
    <actuatorfrc actuator="ankle_x_right"/>
    <actuatorfrc actuator="hip_x_left"/>
    <actuatorfrc actuator="hip_z_left"/>
    <actuatorfrc actuator="hip_y_left"/>
    <actuatorfrc actuator="knee_left"/>
    <actuatorfrc actuator="ankle_y_left"/>
    <actuatorfrc actuator="ankle_x_left"/>
    <actuatorfrc actuator="shoulder1_right"/>
    <actuatorfrc actuator="shoulder2_right"/>
    <actuatorfrc actuator="elbow_right"/>
    <actuatorfrc actuator="shoulder1_left"/>
    <actuatorfrc actuator="shoulder2_left"/>
    <actuatorfrc actuator="elbow_left"/>
  </sensor>

  <keyframe>
    <!--
    The values below are split into rows for readibility:
      torso position
      torso orientation
      spinal
      right leg
      left leg
      arms
    -->
    <key name="squat"
         qpos="0 0 0.596
               0.988015 0 0.154359 0
               0 0.4 0
               -0.25 -0.5 -2.5 -2.65 -0.8 0.56
               -0.25 -0.5 -2.5 -2.65 -0.8 0.56
               0 0 0 0 0 0"/>
    <key name="stand_on_left_leg"
         qpos="0 0 1.21948
               0.971588 -0.179973 0.135318 -0.0729076
               -0.0516 -0.202 0.23
               -0.24 -0.007 -0.34 -1.76 -0.466 -0.0415
               -0.08 -0.01 -0.37 -0.685 -0.35 -0.09
               0.109 -0.067 -0.7 -0.05 0.12 0.16"/>
    <key name="prone"
         qpos="0.4 0 0.0757706
               0.7325 0 0.680767 0
               0 0.0729 0
               0.0077 0.0019 -0.026 -0.351 -0.27 0
               0.0077 0.0019 -0.026 -0.351 -0.27 0
               0.56 -0.62 -1.752
               0.56 -0.62 -1.752"/>
    <key name="supine"
         qpos="-0.4 0 0.08122
               0.722788 0 -0.69107 0
               0 -0.25 0
               0.0182 0.0142 0.3 0.042 -0.44 -0.02
               0.0182 0.0142 0.3 0.042 -0.44 -0.02
               0.186 -0.73 -1.73
               0.186 -0.73 -1.73"/>
  </keyframe>
</mujoco>
"""

モデルを読み込み、制御問題の初期状態として選択した「squat」 [キーフレーム](https://mujoco.readthedocs.io/en/latest/XMLreference.html#keyframe) をレンダリングしましょう。

In [0]:
# モデルの読み込み、データの作成
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# 状態を「squat」キーフレームに設定し、mj_forwardを呼び出す
key = model.key('squat').id
mujoco.mj_resetDataKeyframe(model, data, key)
mujoco.mj_forward(model, data)

# レンダラーが存在する場合は閉じる
if 'renderer' in locals():
  renderer.close()

# レンダラーとカメラの作成
renderer = mujoco.Renderer(model, height=480, width=640)
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 3
camera.elevation = -10

# カメラをヒューマノイドに向けてレンダリング
camera.lookat = data.body('torso').subtree_com
renderer.update_scene(data, camera)
media.show_image(renderer.render())

### 問題定義

以下の最適制御問題を定義します：
- 上記の「squat」キーフレームから開始して、$t=0\ldots T$ の軌道をロールアウトします。
- ロールアウト中に制御入力 $u_t$ が適用されます。これは最初の制御 $u_0$ と最後の制御 $u_T$ の線形補間です。これら2つのベクトルが決定変数 $x = \begin{pmatrix} u_0 & u_T \end{pmatrix}$ です。
- 残差はすべてのタイムステップにわたる以下の連結です：
  - 右手からターゲットへのベクトル。
  - アクチュエータによって加えられるトルク（ある係数でスケーリング。数値的に手とターゲットの距離よりはるかに大きいため）。

残差では `mujoco.rollout` を使用して並列軌道を評価します：

In [0]:
def reach(ctrl0T, target, T, torque_scale, traj=None):
  """ターゲット到達タスクの残差。

  引数:
    ctrl0T: 最初と最後の制御ベクトルの連結。
    target: 右手が到達すべきターゲット。
    T: ロールアウトの終了時間。
    torque_scale: トルクをスケーリングする係数。
    traj: 記録される位置のオプションのリスト。

  戻り値:
    ターゲット到達タスクの残差。
  """
  # 最初と最後のctrlベクトルを抽出し、行ベクトルに転置
  ctrl0 = ctrl0T[:model.nu, :].T
  ctrlT = ctrl0T[model.nu:, :].T

  # mocapボディをターゲットに移動
  mocapid = model.body('target').mocapid
  data.mocap_pos[mocapid] = target

  # mocapターゲットを制御に追加
  nbatch  = ctrl0.shape[0]
  mocap = np.tile(data.mocap_pos[mocapid], (nbatch, 1))
  ctrl0 = np.hstack((ctrl0, mocap))
  ctrlT = np.hstack((ctrlT, mocap))

  # 制御仕様を定義（ctrl + mocap_pos）
  mjtState = mujoco.mjtState
  control_spec = mjtState.mjSTATE_CTRL | mjtState.mjSTATE_MOCAP_POS

  # 制御列を補間してスタック
  nstep = int(np.round(T / model.opt.timestep))
  control = np.stack(np.linspace(ctrl0, ctrlT, nstep), axis=1)

  # 「squat」キーフレームにリセットし、初期状態を取得
  key = model.key('squat').id
  mujoco.mj_resetDataKeyframe(model, data, key)
  spec = mjtState.mjSTATE_FULLPHYSICS
  nstate = mujoco.mj_stateSize(model, spec)
  state = np.empty(nstate)
  mujoco.mj_getState(model, data, state, spec)

  # ロールアウトを実行（sensors.shape == nbatch, nstep, nsensordata）
  states, sensors = rollout.rollout(model, data, state, control,
                                    control_spec=control_spec)

  # 要求された場合、qposをtrajに抽出
  if traj is not None:
    assert states.shape[0] == 1
    # stateの最初の要素（mjData.time）をスキップ
    traj.extend(np.split(states[0, :, 1:model.nq+1], nstep))

  # トルクセンサーをスケーリング
  sensors[:, :, 3:] *= torque_scale

  # センサー値をスタックするためにリシェイプし、列ベクトルに転置
  sensors = sensors.reshape((sensors.shape[0], -1)).T

  # ノーマライザーはTやタイムステップを変更した際に目的関数値を同程度に保つ
  normalizer = 100 * model.opt.timestep / T
  return normalizer * sensors

残りの問題定義を示します：
1. 軌道は0.7秒間積分されます。
2. トルクは0.003でスケーリングされます。
3. 決定変数は `mjData.ctrl` の2つのコピーであるため、制約は `mjData.actuator_ctrlrange` の2つの連結されたコピーです。
4. 初期推定値 $x_0 = \begin{pmatrix} u_0 & u_T \end{pmatrix} = \begin{pmatrix} q_\textrm{squat} & q_\textrm{stand} \end{pmatrix}$ はスクワット姿勢での関節角度に続いてデフォルト（立位）での角度です。位置アクチュエータは角度のセマンティクスを持つため、制御の初期化に角度を使用できます。


In [0]:
T = 0.7               # ロールアウト長（秒）
torque_scale = 0.003  # トルクのスケーリング

# 制約はスタックされた制御の制約
lower = np.atleast_2d(model.actuator_ctrlrange[:,0]).T
upper = np.atleast_2d(model.actuator_ctrlrange[:,1]).T
bounds = [np.vstack((lower, lower)), np.vstack((upper, upper))]

# 初期推定値は制約の中点
x0 = 0.5 * (bounds[1] + bounds[0])

フレームをレンダリングするためのユーティリティ関数を定義し、初期推定値を可視化しましょう：

In [0]:
def render_solution(x, target):
  # reachに位置をtrajに保存するよう依頼
  traj = []
  reach(x, target, T, torque_scale, traj=traj);

  frames = []
  counter = 0
  print('フレームをレンダリング中:', flush=True, end='')
  for qpos in traj:
    # 位置を設定し、mj_forwardを呼び出して運動学を更新
    data.qpos = qpos
    mujoco.mj_forward(model, data)

    # レンダリングしてフレームを保存
    camera.lookat = data.body('torso').subtree_com
    renderer.update_scene(data, camera)
    pixels = renderer.render()
    frames.append(pixels)
    counter += 1
    if counter % 10 == 0:
      print(f' {counter}', flush=True, end='')
  return frames

# デフォルトのターゲットを使用
target = data.mocap_pos[model.body('target').mocapid]

# 初期推定値を可視化
media.show_video(render_solution(x0, target))

### 到達タスクの解

あるターゲットに対して一度解いて、最適化の出力を見てみましょう：

In [0]:
target = (.4, -.3, 1.2)

reach_target = lambda x: reach(x, target, T, torque_scale, traj=None)

r0 = reach_target(x0)
print(f'決定変数xのサイズ: {x0.size}')
print(f'残差r(x)のサイズ: {r0.size}\n')

x, _ = minimize.least_squares(x0, reach_target, bounds);

この解がどのように見えるか確認しましょう：

In [0]:
media.show_video(render_solution(x, target))

複数のターゲット値に対してこれを再実行し、すべてのビデオを作成しましょう：

In [0]:
targets = [(0.4, 0., 0.), (0.2, -1., 0.5), (-1., -.3, 1.), (0., -.2, 2.2)]

frames = []
for target in targets:
  res_target = lambda x: reach(x, target, T, torque_scale)
  print(f'ターゲット {target} に対して最適化中', flush=True)
  x, trace = minimize.least_squares(x0, res_target, bounds,
                                    verbose=minimize.Verbosity.FINAL)
  frames += render_solution(x, target)
  print('\n')

print('ビデオを作成中', flush=True)
media.show_video(frames)

### 非2次ノルム

背景セクションで説明したように、最小二乗法は2次以外のノルムに一般化できます。ここでは非2次ノルムの定義方法を示します。これは推定やシステム同定の文脈で重要であり、裾の重い外乱除去分布は非2次関数の指数に比例します。

タスクの残差、つまり手からターゲットへのベクトルを「Smooth L2」関数 $c(r)$ で評価したいとしましょう。ある平滑化半径 $d \gt 0$ に対して、
$$
c(r) = \sqrt{r^T\cdot r + d^2 } - d
$$
この関数は原点の $d$ サイズの近傍で2次的であり、その後L2ノルムのように線形に増加します。1次および2次導関数は
$$
\begin{align}
s&=\sqrt{r^T\cdot r + d^2 }\\
g &= \tfrac{\partial c}{\partial r} = \frac{r}{s} \\
H &=\tfrac{\partial^2 c}{\partial r^2} = \frac{I_{n_r} - g\cdot g^T}{s}
\end{align}
$$
この最適化タスクにこのノルムを使用する特に良い理由はありませんが、あくまで例示です。

`minimize.Norm` クラスのドキュメントを読みましょう：


In [0]:
print(minimize.Norm.__doc__)

センサーは、手からオブジェクトへのベクトルの3つの `r_pos` 残差値に続く21個の `r_torque` アクチュエータトルクで、合計 `ns = 24` センサーです。これらは軌道全体にわたって連結され、サイズ `24*N`（`N` は軌道のタイムステップ数）の残差になります。適切にリシェイプとスライスを行った後のノルムの実装は以下の通りです：

In [0]:
class SmoothL2(minimize.Norm):
  def __init__(self):
    self.n = model.nsensordata  # 24に等しい
    self.d = 0.1  # 平滑化半径（長さ）

  def value(self, r):
    rr = r.reshape((self.n, -1), order='F')
    r_pos = rr[:3, :]
    s = np.sqrt(np.sum(r_pos**2, axis=0) + self.d**2)
    y_pos = (s - self.d).sum()
    r_torque = rr[3:, :]
    y_torque = 0.5 * (r_torque.T**2).sum()
    return y_pos + y_torque

  def grad_hess(self, r, proj):
    rr = r.reshape((self.n, -1), order='F')
    r_pos = rr[:3, :]
    s = np.sqrt(np.sum(r_pos**2, axis=0) + self.d**2)
    g_pos = r_pos / s
    g_torque = rr[3:, :]
    g = np.vstack((g_pos, g_torque))
    grad = proj.T @ g.reshape((-1, 1), order='F')
    h_proj = proj.copy()  # ノルムのヘッセ行列 * 射影行列
    for i in range(g_pos.shape[1]):
      h_i = (np.eye(3) - g_pos[:, i:i+1] @ g_pos[:, i:i+1].T) / s[i]
      j = self.n*i
      h_proj[j:j+3, :] = h_i @ proj[j:j+3, :]
    hess = proj.T @ h_proj
    return grad, hess

最適化を実行する前に、`least_squares` にノルムの実装をチェックしてもらいましょう。巨大な行列の作成を避けるため、短い軌道シミュレーション時間 `T` を使用します。

In [0]:
target = (.4, -.3, 1.2)

T_short = 0.02

reach_target = lambda x: reach(x, target, T=T_short,
                               torque_scale=torque_scale, traj=None)

x, _ = minimize.least_squares(x0, reach_target, bounds, norm=SmoothL2(),
                              max_iter=1, check_derivatives=True);

実装に自信が持てたので、解がどのように見えるか確認しましょう：

In [0]:
target = (.4, -.3, 1.2)

reach_target = lambda x: reach(x, target, T, torque_scale, traj=None)

x, _ = minimize.least_squares(x0, reach_target, bounds, norm=SmoothL2());

media.show_video(render_solution(x, target))